In [1]:
import mcnp_input_reader as mir

In [2]:
import re

In [3]:
input_path = '/home/giovanni/lavoro/ITER/ITER_lower_port_14_sector7/E-lite_R200430'

In [4]:
mcnp_input = mir.read_file(inputfile)

NameError: name 'inputfile' is not defined

In [7]:
u10043 = mcnp_input.extract(lambda cell: cell.universe_id == 10043)

In [4]:
def finditer_with_line_numbers(pattern, string, flags=0):
    '''
    A version of 're.finditer' that returns '(match, line_number)' pairs.
    '''

    matches = list(re.finditer(pattern, string, flags))
    if not matches:
        return []

    end = matches[-1].end()
    # -1 so a failed 'rfind' maps to the first line.
    newline_table = {-1: 0}
    for i, m in enumerate(re.finditer(r'\n', string), 1):
        # don't find newlines past our last match
        offset = m.start()
        if offset > end:
            break
        newline_table[offset] = i

    # Failing to find the newline is OK, -1 maps to 0.
    for m in matches:
        newline_offset = string.rfind('\n', 0, m.start())
        newline_offset_end = string.rfind('\n', 0, m.end())
        line_number = newline_table[newline_offset]
        line_number_end = newline_table[newline_offset_end]
        yield (m, line_number, line_number_end)

In [5]:
elite_input = mir.read_file(input_path)
first_cell_line = elite_input.cells.get_start_end_lines()[0][0]
elite_lines = elite_input.lines 
headings_lines = []
for line in elite_lines:
    if 'MAIN HEADINGS' in line:
        headings_lines.append(line)
    elif headings_lines:
        if line.upper().startswith('C'):
            headings_lines.append(line)
        else:
            break
headings_lines = ''.join(headings_lines)


In [57]:
heading_sep_pattern = re.compile(r"^C\s+-+.", re.DOTALL|re.MULTILINE)
elite_headings = re.search(heading_sep_pattern, headings_lines)
heading_sep = elite_headings.group()
headers_list = headings_lines.split(elite_headings.group())[1:-1]

In [58]:
print(headers_list)

['\nC                   ENVELOPE STRUCTURE OF SECTOR 1\nC                          PORTS: 1 - 2 - 3\nC                            (+30 degrees)\n', '\nC                 ENVELOPE STRUCTURE OF SECTORS 2 and 3\nC                              NBI SECTOR\nC                       PORTS: 3 - 4 - 5 - 6 - 7\n', '\nC                   ENVELOPE STRUCTURE OF SECTOR 4\nC                          PORTS: 7 - 8 - 9\nC                            (+150 degrees)\n', '\nC                   ENVELOPE STRUCTURE OF SECTOR 5\nC                          PORTS: 9 - 10 - 11\nC                            (-170 degrees)\n', '\nC                   ENVELOPE STRUCTURE OF SECTOR 6\nC                          PORTS: 11 - 12 - 13\nC                            (-130 degrees)\n', '\nC                   ENVELOPE STRUCTURE OF SECTOR 7\nC                          PORTS: 13 - 14 - 15\nC                            (-90 degrees)\n', '\nC                   ENVELOPE STRUCTURE OF SECTOR 8\nC                          PORTS: 15 - 16 

In [60]:
for head in headers_list:
    print(head.strip())

C                   ENVELOPE STRUCTURE OF SECTOR 1
C                          PORTS: 1 - 2 - 3
C                            (+30 degrees)
C                 ENVELOPE STRUCTURE OF SECTORS 2 and 3
C                              NBI SECTOR
C                       PORTS: 3 - 4 - 5 - 6 - 7
C                   ENVELOPE STRUCTURE OF SECTOR 4
C                          PORTS: 7 - 8 - 9
C                            (+150 degrees)
C                   ENVELOPE STRUCTURE OF SECTOR 5
C                          PORTS: 9 - 10 - 11
C                            (-170 degrees)
C                   ENVELOPE STRUCTURE OF SECTOR 6
C                          PORTS: 11 - 12 - 13
C                            (-130 degrees)
C                   ENVELOPE STRUCTURE OF SECTOR 7
C                          PORTS: 13 - 14 - 15
C                            (-90 degrees)
C                   ENVELOPE STRUCTURE OF SECTOR 8
C                          PORTS: 15 - 16 - 17
C                            (-50 degrees)
C          

In [22]:
import re
re_pattern = re.compile(r"C\s+Title\s+:.*?^C\s+Transforms", re.DOTALL|re.MULTILINE)

In [24]:
csv_gen = [row for row in elite_comments]

In [105]:
lista = []
for i, row in enumerate(csv_gen):
    if i < len(csv_gen) - 1:
        lista.append([row[2],csv_gen[i+1][1], row[1],csv_gen[i+1][1]])

In [81]:
csv_gen[0][]

(<re.Match object; span=(300169, 301284), match='C Title         : BM01_C03\nC Author        : A. >,
 8511,
 8531)

In [109]:
lines2 = []
for l in lista:
    #append_line = []
    if all(row.upper().startswith('C') for row in u10043.lines[l[2]:l[1]]):
        lines2.append([l[2],l[1]])
    #for row in u10043.lines[l[0]:l[1]]:
    #    if row[0].upper() == 'C':
    #        append_line.append(False)
    #    else:
    #        append_line.append(True)
    #if not any(append_line):
    #    lines.append([l[2],l[3]])
    

In [78]:
def all_lines_start_with_c(lines, list_range_lines):
    list_c_line_numbers = []
    for range_line in list_range_lines:
        if all(row.upper().startswith('C') for row in lines[range_lines[0]:range_lines[1]]):
            list_c_line_numbers.append(range_line)
    return list_c_line_numbers

True

In [69]:
lines = []
for l in lista:
    append_line = []
    #if  any(not row.upper().startswith('C') for row in u10043.lines[l[0]:l[1]]):
    #    lines.append([l[2],l[3]])
    for row in u10043.lines[l[0]:l[1]]:
        if row[0].upper() == 'C':
            append_line.append(False)
        else:
            append_line.append(True)
    if not any(append_line):
        lines.append([l[2],l[3]])

In [110]:
[l for l in lines if l not in lines2]

[[63782, 97151], [126628, 132981]]

In [108]:
lines2

[[8511, 8554],
 [8554, 8597],
 [8597, 8639],
 [8639, 8681],
 [8681, 8723],
 [8723, 8767],
 [8767, 8809],
 [8809, 8852],
 [8852, 8894],
 [8894, 8936],
 [8936, 8978],
 [8978, 9022],
 [9022, 9756],
 [9756, 10520],
 [10520, 10742],
 [10742, 10782],
 [10782, 10822],
 [10822, 10867],
 [10867, 10907],
 [10907, 10947],
 [10947, 10987],
 [10987, 11027],
 [11027, 11068],
 [11068, 11110],
 [11110, 11146],
 [11146, 11186],
 [11186, 11226],
 [11226, 11266],
 [11266, 11306],
 [11306, 11351],
 [11351, 11396],
 [11396, 11432],
 [11432, 11476],
 [11476, 11498],
 [11498, 11681],
 [11681, 11713],
 [11713, 11758],
 [11758, 11827],
 [11827, 11857],
 [11857, 11879],
 [11879, 11901],
 [11901, 11923],
 [11923, 11944],
 [11944, 11973],
 [11973, 11995],
 [11995, 12017],
 [12017, 12038],
 [12038, 12059],
 [12059, 12080],
 [12080, 12214],
 [12214, 12282],
 [12282, 12312],
 [12312, 12329],
 [12329, 12360],
 [12360, 12391],
 [12391, 12422],
 [12422, 12453],
 [12453, 12484],
 [12484, 12516],
 [12516, 12538],
 [12538

In [103]:
[(i+63782,l) for i, l in enumerate(u10043.lines[63782:97151]) if not l.upper().startswith('C') ]

[(84318, '\n')]

In [55]:
import itertools
iter_tools_line_numers = itertools.chain.from_iterable([*map(range, *zip(*lines))])
dict_line_numbers_itertools = {n: False for n in iter_tools_line_numers }

In [56]:
lines = []

for n, line in enumerate(u10043.lines):
    if dict_line_numbers_itertools.get(n, True):
        lines.append(line)

In [57]:
with open('u10043_2', 'w') as f:
    f.writelines(lines)

In [10]:
BM16BL_lower_block = [380437, 380438, 380439, 380440, 380441, 380442, 380443, 380444, 380445, 380446, 380447, 380448, 380449, 380450, 380451, 380452, 380453, 380454, 380455, 380456, 380457, 380458, 380459, 380460, 380461, 380462, 380463, 380464, 380465, 380466, 380467, 380468, 380469, 380470, 380471, 380472, 380473, 380474, 380475, 380476, 380477, 380478, 380479, 380480, 380481, 380482, 380483, 380484, 380485, 380486, 380487, 380488, 380489, 380490, 380491, 380492, 380493, 380494, 380495, 380496, 380497, 380498, 380499, 380500, 380501, 380502, 380503, 380504, 380505, 380506, 380507, 380508, 380509, 380510, 380511, 380512, 380513, 380514, 380515, 380516, 380517, 380518, 380519, 380520, 380521, 380522, 380523, 380524, 380525, 380526, 380527, 380528, 380529, 380530, 380531, 380532, 380533, 380534, 380535, 380536, 380537, 380538, 380539, 380540, 380541, 380542, 380543, 380544, 380545, 380546, 380547, 380548, 380549, 380550, 380551, 380552, 380553, 380554, 380555, 380556, 380557, 380558, 380559, 380560, 380561, 380562, 380563, 380564, 380565, 380566, 380567, 380568, 380569, 380570, 380571, 380572, 380573, 380574, 380575, 380576, 380577, 380578, 380579, 380580, 380581, 380582, 380583, 380584, 380585, 380586, 380587, 380588, 380589, 380590, 380591, 380592, 380593, 380594, 380595, 380596, 380597, 380598, 380599, 380600, 380601, 380602, 380603, 380604, 380605, 380606, 380607, 380610, 380611, 380612, 380613, 380614, 380615, 380616, 380617, 380618, 380619, 380620, 380621, 380622, 380623, 380624, 380625, 380626, 380627, 380628, 380629, 380630, 380631, 380632, 380633, 380634, 380635, 380636, 380637, 380638, 380639, 380640, 380641, 380642, 380643, 380644, 380645, 380646, 380648, 380647, 381668, 381242, 381254, 381246, 381247, 381243, 381669, 381250, 381255, 381670, 381671, 381672, 381674, 381675, 381676, 381677, 381678]

BM15NDL_mid_block = [370270, 370271, 370272, 370273, 370274, 370275, 370276, 370277, 370278, 370279, 370280, 370281, 370282, 370283, 370284, 370285, 370286, 370287, 370288, 370289, 370290, 370291, 370292, 370293, 370294, 370295, 370296, 370297, 370298, 370299, 370300, 370301, 370302, 370303, 370304, 370305, 370306, 370307, 370308, 370309, 370310, 370311, 370312, 370313, 370314, 370315, 370316, 370317, 370318, 370319, 370320, 370321, 370322, 370323, 370324, 370325, 370326, 370327, 370328, 370329, 370330, 370331, 370332, 370333, 370334, 370335, 370336, 370337, 370338, 370339, 370340, 370341, 370342, 370343, 370344, 370345, 370346, 370347, 370348, 370349, 370350, 370351, 370352, 370353, 370354, 370355, 370356, 370357, 370358, 370359, 370360, 370361, 370363, 370364, 370366, 370367, 370365, 370368, 370370, 370371, 370372, 370373, 370374, 370375, 370376, 370377, 370378, 370379, 370380, 370381, 370382, 370383, 370384, 370385, 370386, 370387, 370388, 370389, 370390, 370391, 370392, 370393, 370394, 370395, 370396, 370397, 370398, 370399, 370400, 373986]

BM14NDL_upper_block = [359351, 359352, 359353, 359354, 359355, 359356, 359357, 359358, 359359, 359360, 359361, 359362, 359363, 359364, 359365, 359366, 359367, 359368, 359369, 359370, 359371, 359372, 359373, 359374, 359375, 359376, 359377, 359378, 359379, 359380, 359381, 359382, 359383, 359384, 359385, 359386, 359387, 359388, 359389, 359390, 359391, 359392, 359393, 359394, 359395, 359396, 359397, 359398, 359399, 359400, 359401, 359402, 359403, 359404, 359405, 359406, 359407, 359408, 359409, 359410, 359411, 359412, 359413, 359414, 359415, 359416, 359417, 359418, 359419, 359420, 359421, 359422, 359423, 359424, 359425, 359426, 359427, 359428, 359429, 359430, 359431, 359432, 359433, 359434, 359435, 359436, 359437, 359438, 359439, 359440, 359441, 359442, 359443, 359444, 359445, 359446, 359447, 359448, 359449, 359450, 359451, 359452, 359453, 359454, 359455, 359456, 359457, 359458, 359459, 359460, 359461, 359462, 359463, 359464, 359465, 359466, 359467, 359468, 359469, 359470, 359471, 359472, 359473, 359474, 359475, 359476, 359477, 359478, 359479, 359480, 359481, 359482, 359483, 359484, 359485, 359486, 359487, 359488, 359489, 359490, 359491, 359492, 359493, 359494, 359495, 359496, 359497, 359498, 359499, 359500, 359501, 359502, 359503, 359504, 359505, 359506, 359507, 359508, 359509, 359510, 359511, 359512, 359513, 359514, 359515, 359516, 359517, 359518, 359519, 359520, 359521, 359522, 359523, 359524, 359525, 359526, 359527, 359528, 359529, 359530, 359531, 359532, 359533, 359534, 359535, 359536, 359537, 359538, 359539, 359540, 359541, 359542, 359543, 359544, 359545, 359546, 360631, 362152, 362153, 362154, 362155, 362156, 362157, 362158, 362159, 362160, 362161, 362162]

In [53]:
mcnp_input.cells.filter(lambda cell: cell.id in BM14NDL_upper_block + BM15NDL_mid_block + BM14NDL_upper_block).get_materials()

{101, 108, 109, 302, 400, 700, 701, 702, 706, 727, 730, 761, 9126}

In [54]:
len([101, 108, 109, 302, 400, 700, 701, 702, 706, 727, 730, 761, 9126])

13

In [12]:
print(mcnp_input.materials[734].input_material_description)

M734
C BM16 S04 SB3 VARIATION, DENSITY CORRECTION: 1.00254
C ***********************************
C *  WATER (126C;4MPA)
C *  MASS DENSITY [G/CC] - 0.9401
C *  VOLUME FRACTION [%] - 7.10
C *  
C ***********************************
C 
C 
         1001.31C 4.46193E-003 $ H  1 AMOUNT(%)  2.0000 AB(%) 99.99
         1002.31C 5.13180E-007 $ H  2 AMOUNT(%)  2.0000 AB(%)  0.01
         8016.31C 2.22580E-003 $ O 16 AMOUNT(%)  1.0000 AB(%) 99.76
C *  
C *  T.A.D. = 6.69366E-003
C *  EFF.DENSITY = 6.67471E-002
C ***********************************
C *  SS316L(N)-IG
C *  MASS DENSITY [G/CC] - 7.93
C *  VOLUME FRACTION [%] - 92.90
C *  
C ***********************************
C 
C 
         6012.31C 1.09628E-004 $ C 12 WEIGHT(%)  0.0300 AB(%) 98.93
        25055.31C 1.61509E-003 $MN 55 WEIGHT(%)  2.0000
        28058.31C 6.43219E-003 $NI 58 WEIGHT(%) 12.5000 AB(%) 68.08
        28060.31C 2.47767E-003 $NI 60 WEIGHT(%) 12.5000 AB(%) 26.22
        28061.31C 1.07703E-004 $NI 61 WEIGHT(%) 12.5000 AB(%)  1

In [9]:
mcnp_input.materials

<MCNPMaterials>: [1, 2]

In [10]:
mcnp_input.materials[1]

MCNPMaterial(id=1, composition='1000. 2 8000. 1', start_line=29, end_line=30, input_material_description='M1 1000. 2\n 8000. 1 $WATER')

## Cell methods

In [11]:
mcnp_input.cells.get_surfaces()

{1, 2, 3, 4, 5, 6, 10, 100, 101, 102}

In [12]:
mcnp_input.cells.get_materials()

{1, 2}

In [13]:
mcnp_input.cells.get_fill_ids()

{1}

In [14]:
mcnp_input.cells.get_universe_ids()

{1}

In [15]:
mcnp_input.cells.get_transformations()

set()

## Filter Capabilities

In [16]:
mcnp_input.cells.filter(lambda cell: cell.mat_id == 1)

id          |mat_id      |density     |imp_n       |imp_p       |universe_id |fill_id     
------------------------------------------------------------------------------------------
1           |1           |-1.0        |0.0         |1.0         |0           |1           
10          |1           |-1.0        |0.0         |1.0         |1           |0           

In [17]:
mcnp_input.cells.filter(lambda cell: cell.mat_id == 2)

id          |mat_id      |density     |imp_n       |imp_p       |universe_id |fill_id     
------------------------------------------------------------------------------------------
100         |2           |-0.001293   |0.0         |1.0         |0           |0           

In [18]:
mcnp_input.cells.filter(lambda cell: cell.id < 20)

id          |mat_id      |density     |imp_n       |imp_p       |universe_id |fill_id     
------------------------------------------------------------------------------------------
1           |1           |-1.0        |0.0         |1.0         |0           |1           
10          |1           |-1.0        |0.0         |1.0         |1           |0           

In [19]:
mcnp_input.cells.filter(lambda cell: cell.start_line < 7)

id          |mat_id      |density     |imp_n       |imp_p       |universe_id |fill_id     
------------------------------------------------------------------------------------------
1           |1           |-1.0        |0.0         |1.0         |0           |1           
10          |1           |-1.0        |0.0         |1.0         |1           |0           
100         |2           |-0.001293   |0.0         |1.0         |0           |0           

In [20]:
mcnp_input.cells.filter(lambda cell: cell.start_line in range(7, 20))

id          |mat_id      |density     |imp_n       |imp_p       |universe_id |fill_id     
------------------------------------------------------------------------------------------
999         |0           |0.0         |0.0         |0.0         |0           |0           

In [21]:
mcnp_input.cells.filter(lambda cell: cell.mat_id == 2).get_surfaces()

{100, 101, 102}

In [22]:
mcnp_input.surfaces.filter(lambda surf: surf.id < 10)

id          |surface_type
-------------------------
1           |PX          
2           |PX          
3           |PY          
4           |PY          
5           |PZ          
6           |PZ          

## Extract method

In [23]:
mcnp_input.extract(lambda cell: cell.mat_id == 2)

Title: Extracted from Water phantom

The input contains:
# of cells: 1
# of surfaces: 3
# of materials: 1
# of transformations: 0

In [24]:
mcnp_input.extract(lambda cell: cell.mat_id == 2).get_start_end_lines() # list of start-stop line numbers

[[6, 8], [22, 23], [23, 24], [24, 25], [31, 33]]

## to_csv method

In [25]:
mcnp_input.cells.filter(lambda cell: cell.mat_id==1).to_csv('cells.csv', ['id','mat_id','density','fill_id'])

In [26]:
csv_cells = 'cells.csv'
!cat $csv_cells